# Using DenseNet with Sun397 dataset

In [1]:
from keras.layers import Input, Dense, Flatten, Dropout
from keras.models import Model
from keras.optimizers import Adam, SGD
from keras.utils import np_utils
from keras import backend as K
import numpy as np
import os
from keras.regularizers import l2
import tensorflow as tf
import time
import datetime
import argparse
import datetime
import socket
import keras 
from sklearn import preprocessing
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
import time
from keras.preprocessing.image import ImageDataGenerator
from PIL import Image, ImageOps
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
##### For one-hot label
from keras.utils import np_utils


/root/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# load raw data

In [ ]:
nb_classes = 397
img_depth = 3
data_dir ='/content/drive/My Drive/Colab Notebooks/SUN_Practice/'+'SUN397'
train_img_file = '/content/drive/My Drive/Colab Notebooks/SUN_Practice/Partitions/Training_01.txt'   # Training_01.txt
test_img_file = '/content/drive/My Drive/Colab Notebooks/SUN_Practice/Partitions/Testing_01.txt'     # Testing_01.txt
classes_name_list = '/content/drive/My Drive/Colab Notebooks/SUN_Practice/Partitions/ClassName.txt'
train_label_file ='/content/drive/My Drive/Colab Notebooks/SUN_Practice/Partitions/ClassName.txt'
test_label_file = '/content/drive/My Drive/Colab Notebooks/SUN_Practice/Partitions/ClassName.txt'

train_img_file_path='/content/drive/My Drive/Colab Notebooks/SUN_Practice/Partitions/Training_01.txt'
test_img_file_path='/content/drive/My Drive/Colab Notebooks/SUN_Practice/Partitions/Testing_01.txt'


# Get Label

In [ ]:
print('Begin to create a map to tansfer the str label to int label...')
class_name_file =  classes_name_list
class_str = [str(line.strip()) for line in open(class_name_file).readlines()]
class_count = len(class_str)
print('%d class names are loaded' % class_count)
        # begin to create the map
le = preprocessing.LabelEncoder()
le.fit(class_str)
print(list(le.classes_))
print('Label map created...')

# Load training data

In [ ]:
desired_img_dim=224

In [ ]:
print('Loading image file %s' % train_img_file_path)

In [ ]:
start_time_ = time.time()
train_img_file_path = [str(line.strip()) for line in open(train_img_file_path).readlines()]  # read all the image file name
nb_sample = len(train_img_file_path)
print('Image count: %d' % nb_sample)

data_resized_holder = np.empty([nb_sample, desired_img_dim, desired_img_dim, img_depth], dtype='float32')

for idx in range(nb_sample):
    img_file1 = train_img_file_path[idx].replace("\\", "/")     # the image file path
    # print(str(img_file1))
    # 1. read the image
    img1 = image.load_img(data_dir+img_file1)

    # 2. resize
    img1 = img1.resize((desired_img_dim, desired_img_dim), resample=0)


    # 6. give to the holder
    data_resized_holder[idx] = img1
    if(idx % 1000==0):
        print('%d image loaded.' % idx)

print('\nImage file loaded, the shape is ' + str(data_resized_holder.shape))


In [ ]:
x_train.shape

**loading the training labels text**

In [ ]:
one_hot = True
print('Loading label file %s' % train_label_file)
label_str = [str(line.strip()) for line in open(train_label_file).readlines()]
nb_unique = len(label_str)
labels_unique = le.transform(label_str)
# print(labels_unique)
labels_holder = np.hstack((  [ labels_unique[i] ] * 50 for i in range(nb_unique)))
# print(labels_holder)
nb_sample = len(labels_holder)
if one_hot == True:
    labels = np.array([[float(i == l) for i in range(nb_classes)] for l in labels_holder])
else:
    labels = labels_holder
print('Labels loaded, shape is:' + str(labels.shape))



**loading training data**

In [ ]:
x_train, nb_train_sample_1 = data_resized_holder, nb_sample

In [ ]:
y_train, nb_train_sample_2 = labels, nb_sample

In [ ]:
del data_resized_holder
del labels

In [ ]:
x_train.shape
y_train.shape

# Load testing data

**Load test labels**

In [ ]:
print('Loading image file %s' % test_img_file_path )
start_time_ = time.time()
test_img_file_path = [str(line.strip()) for line in open(test_img_file_path).readlines()]  # read all the image file name
nb_sample = len(test_img_file_path)
print('Image count: %d' % nb_sample)

data_resized_holder = np.empty([nb_sample, desired_img_dim, desired_img_dim, img_depth], dtype='float32')

for idx in range(nb_sample):
    img_file1 = data_dir + test_img_file_path[idx].replace("\\", "/")     # the image file path
    # print(str(img_file1))
    # 1. read the image
    img1 = image.load_img(img_file1)

    # 2. resize
    img1 = img1.resize((desired_img_dim, desired_img_dim), resample=0)


    # 6. give to the holder
    data_resized_holder[idx] = img1
    if(idx % 1000==0):
        print('%d image loaded.' % idx)

print('\nImage file loaded, the shape is ' + str(data_resized_holder.shape))



**Load test labels**

In [ ]:
one_hot = True
# loading the training labels
print('Loading label file %s' % test_label_file)
label_str = [str(line.strip()) for line in open(test_label_file).readlines()]
nb_unique = len(label_str)
labels_unique = le.transform(label_str)
# print(labels_unique)
labels_holder = np.hstack((  [ labels_unique[i] ] * 50 for i in range(nb_unique)))
# print(labels_holder)
nb_sample = len(labels_holder)
if one_hot == True:
    labels = np.array([[float(i == l) for i in range(nb_classes)] for l in labels_holder])
else:
    labels = labels_holder
print('Labels loaded, shape is:' + str(labels.shape))



In [ ]:
x_test, nb_test_sample_1 = data_resized_holder, nb_sample
y_test, nb_test_sample_2 = labels, nb_sample

In [ ]:
del data_resized_holder
del labels

# Saving training data and testing data to npy.file

**saving x_train and y_trian as npy.file**

In [ ]:
np.save('/content/drive/My Drive/Colab Notebooks/SUN_Practice/x_tain.npy',x_train)

In [ ]:
np.save('/content/drive/My Drive/Colab Notebooks/SUN_Practice/y_train.npy',y_train)

**saving x_train and y_trian as npy.file**

In [ ]:
np.save('/content/drive/My Drive/Colab Notebooks/SUN_Practice/x_test.npy',x_test)

In [ ]:
np.save('/content/drive/My Drive/Colab Notebooks/SUN_Practice/x_test.npy',y_test)

# Load training data and testing data to npy.file

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0";
import numpy as np


import keras
# from keras.applications.vgg16 import VGG16
from keras.applications.densenet import DenseNet121

# **Load train data and test data**

In [2]:
x_train = np.load('/root/Code_GCP/SUN_Practice/x_tain.npy')

In [3]:
y_train = np.load('/root/Code_GCP/SUN_Practice/y_train.npy')

**Load test data**

In [4]:
x_test = np.load('/root/Code_GCP/SUN_Practice/x_test.npy')

In [5]:
y_test = np.load('/root/Code_GCP/SUN_Practice/y_test.npy')

In [6]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(19850, 224, 224, 3)
(19850, 397)
(19850, 224, 224, 3)
(19850, 397)


# **Data Auguation**

In [7]:
generator = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')


In [8]:
from keras.applications.densenet import preprocess_input

x_test = preprocess_input(x_test)
x_train = preprocess_input(x_train)

# Using Densenet network

In [9]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0";


import keras
from keras.applications.densenet import DenseNet121

In [10]:
model = DenseNet121(weights='imagenet', include_top=False, classes=397, input_shape=(224,224,3))
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 112, 112, 64) 9408        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1/conv[0][0]                 
__________________________________________________________________________________________________
conv1/relu

In [11]:
from keras.layers import Flatten, Dense, Dropout
from keras.models import Model
from keras.layers.pooling import GlobalAveragePooling2D

# x = model.get_layer('block5_pool').output
# x = Flatten(name='flatten')(x)
# x = Dense(512, activation='relu', name='fc1')(x)
# x = Dense(4096, activation='relu', name='fc2')(x)
# x = Dropout(0.3)(x)
# x = Dense(397, activation='softmax', name='predictions')(x)
x = model.get_layer('relu').output
x = GlobalAveragePooling2D(name='pool')(x)
x = Dense(397, activation='softmax', name='fc1')(x)

model_updated = Model(inputs=model.input, outputs=x)

In [12]:
from keras import optimizers
from keras.callbacks import EarlyStopping

In [13]:
def learning_rate_schedule(epoch):
    if epoch <= 10:
        return 1e-4 # 0.00001
    elif epoch <= 20:
        return 1e-5
    elif epoch <= 30:
        return 1e-6
    else:
        return 1e-7
    return LR

In [14]:
model_updated.save_weights('model_Dense_places.h5')

In [15]:
batch_size = 32
training_runs = []
for i in range(3):
    #model_updated.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
  

    model_updated.compile(loss='mean_squared_error', optimizer=optimizers.adam(lr=0.0001), metrics=['accuracy'])
    keras.callbacks.LearningRateScheduler(learning_rate_schedule)

    history = model_updated.fit_generator(generator.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch=len(x_train) / batch_size, 
                    epochs=30,
                    verbose=1,
                    shuffle=True ,
                    validation_data=(x_test, y_test))
#     history = model_updated.fit(
#     x_train, y_train,
#     batch_size=64, shuffle=True, epochs=30,
#     validation_data=(x_test, y_test)
#     )
    
   
    training_runs.append(history)
    model_updated.get_weights()
    if i == 2:
        model_updated.save_weights('model_dense_save.h5')
    else:
        model_updated.load_weights('model_Dense_places.h5')
    print()

Epoch 1/30
621/620 [==============================] - 414s 667ms/step - loss: 0.0025 - acc: 0.0292 - val_loss: 0.0025 - val_acc: 0.0643
Epoch 2/30
621/620 [==============================] - 365s 588ms/step - loss: 0.0024 - acc: 0.1062 - val_loss: 0.0023 - val_acc: 0.1604
Epoch 3/30
621/620 [==============================] - 369s 594ms/step - loss: 0.0023 - acc: 0.1963 - val_loss: 0.0022 - val_acc: 0.2335
Epoch 4/30
621/620 [==============================] - 369s 594ms/step - loss: 0.0021 - acc: 0.2806 - val_loss: 0.0021 - val_acc: 0.2918
Epoch 5/30
621/620 [==============================] - 370s 596ms/step - loss: 0.0019 - acc: 0.3547 - val_loss: 0.0020 - val_acc: 0.3460
Epoch 6/30
621/620 [==============================] - 363s 585ms/step - loss: 0.0018 - acc: 0.4141 - val_loss: 0.0019 - val_acc: 0.3801
Epoch 7/30
621/620 [==============================] - 361s 581ms/step - loss: 0.0017 - acc: 0.4665 - val_loss: 0.0019 - val_acc: 0.3997
Epoch 8/30
621/620 [============================

KeyboardInterrupt: 

In [ ]:
# from keras import optimizers
# from keras.callbacks import EarlyStopping

# # model_updated.load_weights('model1.h5')
# training_runs = []
# for i in range(3):
#     if i!=0:
# #         model_updated.load_weights('model_initial.h5')
# #     else:
#         model_updated.load_weights('model_dense.h5')
#     model_updated.compile(loss='categorical_crossentropy', optimizer=optimizers.adam(lr=0.0001), metrics=['accuracy'])
#     keras.callbacks.LearningRateScheduler(learning_rate_schedule)
#     history = model_updated.fit(x_train, y_train, batch_size=128, shuffle=True, epochs=10, validation_data=(x_test, y_test))
#     training_runs.append(history)
#     model_updated.get_weights()
#     model_updated.save_weights('model_dense.h5')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.subplot(2, 3, 1)
plt.plot(training_runs[0].history['acc'])
plt.legend(['train'], loc='lower right')

plt.subplot(2, 3, 2)
plt.plot(training_runs[1].history['acc'])
plt.legend(['train'], loc='lower right')

plt.subplot(2, 3, 3)
plt.plot(training_runs[2].history['acc'])
plt.legend(['train'], loc='lower right')

plt.subplot(2, 3, 4)
plt.plot(training_runs[0].history['val_acc'])
plt.legend(['test'], loc='lower right')

plt.subplot(2, 3, 5)
plt.plot(training_runs[1].history['val_acc'])
plt.legend(['test'], loc='lower right')

plt.subplot(2, 3, 6)
plt.plot(training_runs[2].history['val_acc'])
plt.legend(['test'], loc='lower right')

plt.tight_layout() 

plt.show()



In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
print("Average training accuracy: {}".format(np.mean([training_runs[0].history['acc'][-1], 
                                                     training_runs[1].history['acc'][-1], training_runs[2].history['acc'][-1]])))
print("Average testing accuracy: {}".format(np.mean([training_runs[0].history['val_acc'][-1], 
                                                     training_runs[1].history['val_acc'][-1], training_runs[2].history['val_acc'][-1]])))